# Importing Data

In [ ]:
import pandas as pd

In [ ]:
FILE_PATH = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/csv_files/EN2_STAR_CHR_0102890318_20070206T133547_20070402T070302.csv'

In [ ]:
df = pd.read_csv(FILE_PATH)
df.head()

,DATEBARTT,WHITEFLUXSYS
0,54138.073885,219929.30
1,54138.079811,220816.39
2,54138.085737,220129.64
3,54138.091662,219876.34
4,54138.097588,219744.33


In [ ]:
import numpy as np

y = df.WHITEFLUXSYS.to_numpy()
x = df.DATEBARTT.to_numpy()

# Data normalization

Before applying Butterworth filter, we have to ...
- 1. Create artificial borders
- 2. Multiply the data by $(-1)^{i}$
- 3. Apply padding algorithm

## 1. Artificial borders

Our first step is to add artificial borders to the array. After some studies and test it is notable that, for some reason, when the Butterworth filter is used, it can create some distorcion on the first and last values of the array.

Intend to avoid this unexpected values, we add some points on the begining and at the end of the array, so the distorcion will occur on this artificial borders that we create. Then, we just have to cut them off to return to the original array.

In order to not modify the array too much, the function _artificial_borders_ will add 15 points on the begining that have the same value as the first element, y[0] and, just like at the begining, at the end, we will add 15 points that have the same value as the lastest element, y[-1].

In [ ]:
def artifical_borders(array, num):
  aux_pre = np.zeros(num)
  aux_pos = np.zeros(num)
  i = 0
  for i in range(num):
    aux_pre[i] = array[0]
    aux_pos[i] = array[-1]
  
  return np.concatenate((aux_pre, array, aux_pos)).ravel()

## 2. Multiplying data by $(-1)^{i}$

And then, we must multiply each value of the array by the factor $(-1)^{i}$, where i is the position index. 

Intending to comprehend it, we have to look at some properties of the Fourier transform.

It is know that:
$$
f(x)\cdot e^{i 2 \pi (u_0x / M) } \iff F(u-u_0)  \text{ and }  (x-x_0) \iff F(u)\cdot e^{i 2 \pi (u_0x / M)}
$$


where $\iff$ is a ____


Taking $u_{0} = \text{M}/2$, we have:
$$
e^{i 2 \pi (u_0x / M)} = e^{i 2 \pi (\frac{Mx}{2M})} = e^{i \pi (x)}
$$

By Euler's Identity: $ e^{i \pi} + 1 = 0$, we have:
$$
e^{i 2 \pi (u_0x / M)} = (-1)^{x} 
$$  <br />

Therfore,

$$
f(x)\cdot (-1)^{x} \iff F(u-M/2) \text{ and } f(x-M/2) \iff F(u) \cdot (-1)^{x} 
$$

<br /> Since this properties, we do multiply the date by $(-1)^{i}$ before calculating Fourier transform to centralize all the high frequencies in the center of the graphic, that is, the (0, 0) on frequencies domain appeared on the center and the highest frequencies stay on higher distances to that center.


In [ ]:
def multiplying_by_minus_one_to_index(array):
  i = 0
  new_array = np.ones(len(array))

  for i in range(len(array)):
    new_array[i] = array[i] * ( (-1)**(i) )
  
  return new_array

## 3. Padding

One good practice before filtering data with Butterworth is to apply a procedure called Padding. It consists in add zeros in the arrays util it gets the double of the lenght.


Take the function f, with f $\subset$ A(x, y), padding will be represeted as $f_p$ : <br />

<br />
$$
f_p(x) = \begin{cases}
   f(x) &\text{if } 0\le x \le A-1 \\
   0 &\text{if } A \le x \le P
\end{cases}
$$

<br /> Padding remove the implicit periodicity of the funcion that can appeared on the filtered data.
In other terms, the padding prevents the convolution of two functions from generating unexpected (periodic) results.

In [ ]:
def padding(array):
  return np.append(array, np.zeros(len(array)))

Now, our date is ready to be filtered.

The next step is to calculate the Fourier Transform.

# Fourier Transform


$$ 

\hat{f}(\omega) \equiv F(\omega) \equiv \mathcal{F}\{f(t)\} = \int_{-\infty}^{\infty} f(t)\space e^{-j\space \omega t} dt

$$

where:
- 

In [ ]:
def fourier_transform(array):
  return np.fft.fft(array)

# Butterworth Filter

Finally, we must multiply the results of Fourier transform by the filter's funcion, in this case, by the Butterworth Transfer funcion $ H_{n} (j\space\omega) $.

$$ 

G_{n}(\omega)  = |H_{n}(j\space\omega)| = \frac{1}{ \sqrt{ 1 + \big(\frac{\omega}{\omega_c}\big)^{2n} } } 

$$

where:

- G is the gain of an n-order Butterworth low-pass filter
- H is transfer funcion
- j is imaginary number
- n is the order of the filter
- ω  is the angular frequency [rad/s],
- $\omega_c$ is the cutoff frequency [rad/s].

# Inverse Fourier Transform

To go back to spacial domain, we apply the inverse Fourier Transform, $ f(\omega) $, to the product $ \mathcal{F}\{f(t)\} \cdot H_{n}(j\space\omega) $.

$$ 

f(\omega)  \equiv \mathcal{F}^{-1}\{f(t)\} = \frac{1}{2\pi} \int_{-\infty}^{\infty} f(t)\space e^{i\space \omega t} d\omega

$$

where:
- $\omega = 2\pi f$ is the angular frequency

In [ ]:
def inverse_fourier_transform(array):
  return np.fft.ifft(array)

# Undo the procedure 2. Multiplying data by $(-1)^{i}$

In [ ]:
y1_pos = multiplying_by_minus_one_to_index( # before results )

In [ ]:
x1_pos = # before results

# Undo the procedure 3. Padding

In [ ]:
y2_pos = y1_pos[:int(len(y1_pos)/2)]

In [ ]:
x2_pos = x1_pos[:int(len(x1_pos)/2)]

# Undo the procedure 1. Artificial borders

In [ ]:
param = int(15)

In [ ]:
y3_pos = np.delete(y2_pos, np.s_[:param])
y4_pos = np.delete(y3_pos, np.s_[-param:])

In [ ]:
x3_pos = np.delete(x2_pos, np.s_[:param])
x4_pos = np.delete(x3_pos, np.s_[-param:])